In [1]:
from openai import OpenAI
import os
import json
import re
from key import get_key

### Preparing data

In [2]:
# getting the OPENAI key
client = OpenAI(
    api_key= get_key()
)

In [3]:
# folders for our application versions
""" OLD_XML_DIR = "../layouts/our.v.1"
NEW_XML_DIR = "../layouts/our.v.2"
OLD_TESTS_DIR = "../tests/our.v.1"
NEW_TESTS_DIR = "../tests/our.v.2" """

' OLD_XML_DIR = "../layouts/our.v.1"\nNEW_XML_DIR = "../layouts/our.v.2"\nOLD_TESTS_DIR = "../tests/our.v.1"\nNEW_TESTS_DIR = "../tests/our.v.2" '

In [4]:
# folders for Omni-Notes
OLD_XML_DIR = "../layouts/v6.1.0"
NEW_XML_DIR = "../layouts/v6.3.0"
OLD_TESTS_DIR = "../tests/v6.1.0"
NEW_TESTS_DIR = "../tests/v6.3.0"

In [5]:
# function used to read a content of a file
def read_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

# reading all files from different directories and store their content in a dictionary where the keys are the filenames and the values are the file contents.
old_xml_files = {f: read_file(os.path.join(OLD_XML_DIR, f)) for f in os.listdir(OLD_XML_DIR)}
new_xml_files = {f: read_file(os.path.join(NEW_XML_DIR, f)) for f in os.listdir(NEW_XML_DIR)}
old_test_files = {f: read_file(os.path.join(OLD_TESTS_DIR, f)) for f in os.listdir(OLD_TESTS_DIR)}

In [6]:
# converting XMLs into a single string, for LLM input
old_xml_data = "\n\n".join([f"File: {name}\n{content}" for name, content in old_xml_files.items()])
new_xml_data = "\n\n".join([f"File: {name}\n{content}" for name, content in new_xml_files.items()])

### First LLM call

In [7]:
# first LLM step, finding the XML differences between different GUI versions 
xml_differences = {}

for xml_name, old_xml_content in old_xml_files.items():
    new_xml_content = new_xml_files.get(xml_name, "")

    prompt = f"""
    You are an expert in analyzing Android UI XML layouts.
    Here is the old UI XML layout:
    {old_xml_content}

    Here is the updated UI XML layout:
    {new_xml_content}

    Please list all UI changes between these two versions.
    Format the response as structured text, including:
    - Elements that were **added** (with their IDs and attributes).
    - Elements that were **removed**.
    - Elements that were **renamed**.
    - Any **structural changes** (e.g., button moved inside another layout).
    - Any **attribute changes** (e.g., text color changed).
    - Any **value changes**, checking the text content, checking the text attribute in the every items (e.g., text content changed, for example if the button text changed or the title changed). For example if an item, in the old version, have the attribute "text="Item 1" and in the new version for the same item the attribute is "text="Item 2", then it is a value change, so you have to write that as a difference.

    Return the response in JSON format.
    """

    response = client.chat.completions.create(
        messages=[{"role": "system", "content": "You are an expert in Android UI structure analysis."},
                  {"role": "user", "content": prompt}],
        model="gpt-4o-mini"
    )

    xml_differences[xml_name] = response.choices[0].message.content

    print(f"XML differences detected for {xml_name}")

XML differences detected for screen7.xml
XML differences detected for screen6.xml
XML differences detected for screen5.xml
XML differences detected for screen1.xml
XML differences detected for screen2.xml
XML differences detected for screen3.xml
XML differences detected for screen12.xml
XML differences detected for screen11.xml
XML differences detected for screen10.xml
XML differences detected for screen14.xml
XML differences detected for screen15.xml
XML differences detected for screen8.xml
XML differences detected for screen9.xml


In [8]:
# function used to clean the LLM-generated code, that starts with ```java and end with ```
def clean_java_code(java_code):
    """ Remove the code block markers (```java ... ```) """
    return re.sub(r'```java\n|\n```', '', java_code)

In [9]:
# function used to clean the LLM-generated code, that starts with ```kotlin and end with ```
def clean_kotlin_code(kotlin_code):
    """ Remove the code block markers (```kotlin ... ```) """
    return re.sub(r'```kotlin\n|\n```', '', kotlin_code)

### Second LLM call

In [10]:
# second LLM step, updating the test cases, starting from the old ones, analyzing all the xml changes
for test_name, old_test in old_test_files.items():
    prompt = f"""
    You are an expert in Android UI test automation using Espresso.

    The Android app's UI has changed, and test cases need to be updated accordingly.

    Here are all the detected UI changes:
    {xml_differences}

    Here is an old test case:
    {old_test}

    Please update the test case to ensure it remains valid with the new UI structure.
    Ensure that:
    - It still verifies the original functionality.
    - It interacts with the correct elements based on the updated XML.
    - Any removed elements are handled gracefully.
    - Any renamed or moved elements are adjusted accordingly.
    - It follows best practices for Android UI test automation.

    Return only the updated Java test code without any explanation.
    """

    response = client.chat.completions.create(
        messages=[{"role": "system", "content": "You are an expert in Android UI test automation."},
                  {"role": "user", "content": prompt}],
        model="gpt-4o-mini"
    )

    updated_code = response.choices[0].message.content.strip()
    cleaned_java_code = clean_java_code(updated_code)
    # cleaned_kotlin_code = clean_kotlin_code(updated_code) # this used in case of our GUI, bacause we did our tests in Kotlin

    file_path = f"{NEW_TESTS_DIR}/{test_name}"
    with open(file_path, "w") as f:
        f.write(cleaned_java_code)
        # f.write(cleaned_kotlin_code) # this used in case of our GUI, bacause we did our tests in Kotlin
    print(f"Saved {file_path}")

Saved ../tests/v6.3.0/CategoryLifecycleTest.java
Saved ../tests/v6.3.0/SearchTagBackArrowTest.java
Saved ../tests/v6.3.0/NoteLifecycleTest.java
Saved ../tests/v6.3.0/FabCameraNoteTest.java
Saved ../tests/v6.3.0/AutoBackupTest.java
Saved ../tests/v6.3.0/RecurrenceRuleTest.java
Saved ../tests/v6.3.0/SettingsActivityTest.java
Saved ../tests/v6.3.0/RemindersLifecycleTest.java
Saved ../tests/v6.3.0/BaseEspressoTest.java
Saved ../tests/v6.3.0/MrJingleLifecycleTest.java
Saved ../tests/v6.3.0/NoteListMenuTest.java
Saved ../tests/v6.3.0/DrawerMenusEspressoTest.java
Saved ../tests/v6.3.0/FabLifecycleTest.java


### Evaluation

In [11]:
# folder for the goal test cases
GOAL_TESTS_DIR = "../tests/v6.3.0goal"

In [12]:
# reading all files from different directories and store their content in a dictionary where the keys are the filenames and the values are the file contents.
new_test_files = {f: read_file(os.path.join(NEW_TESTS_DIR, f)) for f in os.listdir(NEW_TESTS_DIR)}
goal_test_files = {f: read_file(os.path.join(GOAL_TESTS_DIR, f)) for f in os.listdir(GOAL_TESTS_DIR)}

In [13]:
from collections import Counter

# function used to compute precision and recall between AI-generated and human-written test cases
def compute_precision_recall(ai_test, human_test):
    
    # function used to tokenize Java code by splitting words, removing special characters, and ignoring case
    def tokenize(code):
        return re.findall(r'\w+', code.lower())  # Extract words (ignoring special chars)
    
    ai_tokens = tokenize(ai_test)
    human_tokens = tokenize(human_test)

    ai_counter = Counter(ai_tokens)
    human_counter = Counter(human_tokens)

    # compute intersection: words that appear in both tests
    common_tokens = sum((ai_counter & human_counter).values())

    precision = common_tokens / sum(ai_counter.values()) if ai_counter else 0
    recall = common_tokens / sum(human_counter.values()) if human_counter else 0

    return precision, recall, ai_counter, human_counter

# function used to compute overall precision and recall across all test cases
def compute_overall_precision_recall(total_ai_counter, total_human_counter, total_common_tokens):
    overall_precision = total_common_tokens / sum(total_ai_counter.values()) if total_ai_counter else 0
    overall_recall = total_common_tokens / sum(total_human_counter.values()) if total_human_counter else 0
    return overall_precision, overall_recall

total_ai_counter = Counter()
total_human_counter = Counter()
total_common_tokens = 0

# loop through all test cases and compare AI-generated vs. human-written tests
test_count = 0
for test_name, ai_test in new_test_files.items():
    if test_name not in goal_test_files:
        print(f"Goal test for {test_name} not found, skipping...")
        continue  

    goal_test = goal_test_files[test_name]

    precision, recall, ai_counter, human_counter = compute_precision_recall(ai_test, goal_test)
    
    print(f"Test: {test_name}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("------------------------------------")

    total_ai_counter.update(ai_counter)
    total_human_counter.update(human_counter)
    total_common_tokens += sum((ai_counter & human_counter).values())

    test_count += 1

overall_precision, overall_recall = compute_overall_precision_recall(
    total_ai_counter, total_human_counter, total_common_tokens
)

print("----- OVERALL METRICS -----")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print("---------------------------")

Test: CategoryLifecycleTest.java
Precision: 0.9987
Recall: 0.8588
------------------------------------
Test: SearchTagBackArrowTest.java
Precision: 0.8596
Recall: 0.4558
------------------------------------
Test: NoteLifecycleTest.java
Precision: 0.9790
Recall: 0.6619
------------------------------------
Test: FabCameraNoteTest.java
Precision: 1.0000
Recall: 0.6452
------------------------------------
Test: AutoBackupTest.java
Precision: 1.0000
Recall: 0.8624
------------------------------------
Test: RecurrenceRuleTest.java
Precision: 0.9956
Recall: 0.6338
------------------------------------
Test: SettingsActivityTest.java
Precision: 0.9932
Recall: 0.8420
------------------------------------
Test: RemindersLifecycleTest.java
Precision: 1.0000
Recall: 0.6254
------------------------------------
Test: BaseEspressoTest.java
Precision: 0.9914
Recall: 0.7670
------------------------------------
Test: MrJingleLifecycleTest.java
Precision: 0.9699
Recall: 0.7655
-----------------------------